In [1]:
from power import *
from power.systems import *
from opf_linear.opf_loss import LinearDispatch
from utils.load_scen import apply_load_scen
from utils.wnd_scen import apply_wnd_scen
import numpy as np
from opf_linear.utils.extr_and_save import extract_and_save_results
from pathlib import Path 
import copy
import pandas as pd
import re
import json

In [2]:
nets = [B6L8EOL(), IEEE118EOL(), B3EOL()]

In [3]:
# --- Listas para armazenar CADA dado de TODOS os arquivos ---
data_lists = {
    "sumario": [],
    "cargas_individuais": [],
    "perdas_por_barra": [],
    "curtailment_por_gerador": [],
    "geracao": [],
    "thetas": [],
    "fluxo": [],
    "lmp": [],
    "congestionamento_fluxo": [],
    "limites_geracao": []
}

print("Iniciando extração COMPLETA. Nenhum dado será deixado para trás.")

# 1. Itera sobre as redes
for net in nets:
    system_path = Path(f"results_{net.name}")
    system_name = net.name
    
    if not system_path.is_dir():
        print(f"Aviso: Diretório não encontrado para '{system_name}'. Pulando.")
        continue

    # 2. Itera sobre os cenários e arquivos JSON
    for file_path in system_path.rglob('*.json'):
        match = re.search(r'scenario_(\d+)_(ctg_(\w+)|(BASE_CASE))\.json', file_path.name)
        if not match:
            continue
        
        scenario_num = int(match.group(1))
        contingency_name = match.group(3) if match.group(3) else match.group(4)
        
        run_keys = {'sistema': system_name, 'cenario': scenario_num, 'contingencia': contingency_name}

        with open(file_path, 'r') as f:
            try:
                data = json.load(f)

                # --- Extração exaustiva ---
                # Nível Raiz e Sumários
                data_lists["sumario"].append({**run_keys, 'solver_status': data.get('solver_status'), 'custo_total': data.get('custo_total'), 'carga_total_pu': data.get('sumario_perdas', {}).get('carga_total_pu'), 'perdas_totais_pu': data.get('sumario_perdas', {}).get('perdas_totais_pu'), 'curtailment_total_pu': data.get('sumario_curtailment', {}).get('curtailment_total_pu')})
                
                # sumario_perdas
                for barra, valor in data.get('sumario_perdas', {}).get('cargas_individuais_pu', {}).items():
                    data_lists["cargas_individuais"].append({**run_keys, 'barra_id': barra, 'carga_pu': valor})
                for barra, valor in data.get('sumario_perdas', {}).get('perdas_por_barra_pu', {}).items():
                    data_lists["perdas_por_barra"].append({**run_keys, 'barra_id': barra, 'perda_pu': valor})

                # sumario_curtailment
                for gen, valores in data.get('sumario_curtailment', {}).get('curtailment_por_gerador', {}).items():
                    data_lists["curtailment_por_gerador"].append({**run_keys, 'gerador_id': gen, **valores})
                
                # primal_results
                for gen, valor in data.get('primal_results', {}).get('geracao_pu', {}).items():
                    data_lists["geracao"].append({**run_keys, 'gerador_id': gen, 'geracao_pu': valor})
                for barra, valor in data.get('primal_results', {}).get('thetas_deg', {}).items():
                    data_lists["thetas"].append({**run_keys, 'barra_id': barra, 'theta_deg': valor})
                for linha, valor in data.get('primal_results', {}).get('fluxo_pu', {}).items():
                    data_lists["fluxo"].append({**run_keys, 'linha_id': linha, 'fluxo_pu': valor})

                # dual_results
                for barra, valor in data.get('dual_results', {}).get('custo_marginal_de_energia_LMP', {}).items():
                    data_lists["lmp"].append({**run_keys, 'barra_id': barra, 'lmp': valor})
                for linha, valores in data.get('dual_results', {}).get('congestionamento_de_fluxo', {}).items():
                    data_lists["congestionamento_fluxo"].append({**run_keys, 'linha_id': linha, **valores})
                for gen, valores in data.get('dual_results', {}).get('limites_de_geracao', {}).items():
                    data_lists["limites_geracao"].append({**run_keys, 'gerador_id': gen, **valores})

            except (json.JSONDecodeError, AttributeError) as e:
                print(f"Erro ao processar o arquivo {file_path}: {e}")

print("Extração finalizada. Criando e limpando todos os DataFrames...")

# --- Criação e Limpeza dos DataFrames ---
dfs = {}
for name, data_list in data_lists.items():
    if not data_list:
        print(f"- Nenhuma informação encontrada para '{name}'.")
        continue
    
    df = pd.DataFrame(data_list)
    
    # Limpeza de colunas comuns
    for col in df.columns:
        if 'id' in col:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if df[col].dtype == 'object':
            # Tenta converter para numérico, tratando erros
            df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')

    dfs[name] = df
    print(f"+ DataFrame '{name}' criado com {len(df)} linhas.")

print(f"\nProcesso concluído! {len(dfs)} DataFrames foram gerados no dicionário 'dfs'.")
print(f"Use dfs['nome_do_df'] para acessar. Nomes disponíveis: {list(dfs.keys())}")

# Exemplo de como acessar e ver a estrutura de um dos novos DFs:
if 'congestionamento_fluxo' in dfs:
    print("\n--- Exemplo: Informações do DataFrame 'congestionamento_fluxo' ---")
    dfs['congestionamento_fluxo'].info()
    print("\n--- Primeiras 5 linhas de 'congestionamento_fluxo' ---")
    print(dfs['congestionamento_fluxo'].head())

Iniciando extração COMPLETA. Nenhum dado será deixado para trás.
Extração finalizada. Criando e limpando todos os DataFrames...
+ DataFrame 'sumario' criado com 2000 linhas.


C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.

+ DataFrame 'cargas_individuais' criado com 170660 linhas.


C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.

+ DataFrame 'perdas_por_barra' criado com 221320 linhas.
+ DataFrame 'curtailment_por_gerador' criado com 20880 linhas.


...

C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')


+ DataFrame 'lmp' criado com 221320 linhas.


C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')


+ DataFrame 'congestionamento_fluxo' criado com 346690 linhas.
+ DataFrame 'limites_geracao' criado com 292830 linhas.

Processo concluído! 10 DataFrames foram gerados no dicionário 'dfs'.
Use dfs['nome_do_df'] para acessar. Nomes disponíveis: ['sumario', 'cargas_individuais', 'perdas_por_barra', 'curtailment_por_gerador', 'geracao', 'thetas', 'fluxo', 'lmp', 'congestionamento_fluxo', 'limites_geracao']

--- Exemplo: Informações do DataFrame 'congestionamento_fluxo' ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346690 entries, 0 to 346689
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   sistema          346690 non-null  object 
 1   cenario          346690 non-null  int64  
 2   contingencia     346690 non-null  object 
 3   linha_id         346690 non-null  int64  
 4   limite_superior  346690 non-null  float64
 5   limite_inferior  346690 non-null  float64
dtypes: float64(2), int64(2), object(2)

C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')
C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\1801271416.py:91: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace('$', '', regex=False).str.replace('/pu', '', regex=False), errors='ignore')


In [8]:
# Pega o DataFrame de sumário
sumario_df = dfs['sumario']

# CORREÇÃO: Usamos .apply() para aplicar a função nlargest a cada grupo
piores_custo_por_sistema = sumario_df.groupby('sistema', group_keys=False).apply(lambda x: x.nlargest(5, 'custo_total'))

print("As 5 piores contingências por CUSTO TOTAL (separado por sistema):")
print(piores_custo_por_sistema)

As 5 piores contingências por CUSTO TOTAL (separado por sistema):
             sistema  cenario contingencia solver_status  custo_total  \
1999        B3_EOLIC        9            2       Optimal      2904.01   
1983        B3_EOLIC        5            2       Optimal      1957.53   
1991        B3_EOLIC        7            2       Optimal      1590.42   
1975        B3_EOLIC        3            2       Optimal       674.26   
1990        B3_EOLIC        7            1       Optimal       615.75   
51        B6L8_EOLIC        5            5       Optimal     10503.08   
15        B6L8_EOLIC        1            5       Optimal     10332.32   
24        B6L8_EOLIC        2            5       Optimal     10272.07   
16        B6L8_EOLIC        1            6       Optimal      9657.79   
6         B6L8_EOLIC        0            5       Optimal      9638.15   
1691  IEEE_118_Eolic        8           25       Optimal    366300.30   
1609  IEEE_118_Eolic        8          118       Optimal  

C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_18024\844435495.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  piores_custo_por_sistema = sumario_df.groupby('sistema', group_keys=False).apply(lambda x: x.nlargest(5, 'custo_total'))
